# Qlunc's presentation

Qlunc stands for **Q**uantification of **l**idar **unc**ertainty. It is stored as a public SWE [repository](https://github.com/SWE-UniStuttgart/Qlunc).

1. Install [anaconda](https://anaconda.org/anaconda/conda)

2. Download or [clone](https://github.com/SWE-UniStuttgart/Qlunc.git) Qlunc´s repository

3. Create an environment and install dependencies

Anaconda prompt:

> conda env create -f environment.yml

> conda activate <environment name> (by default Qlunc_Env)
    

    

## Creating objects as python classes (*Qlunc_Classes.py*)

The goal is to create a lidar digital twin to assess the uncertainty of an actual one, before it is built. First we create the lidar components by using python classes. We give some parameter values to these classes, characterizing the components, inculding a function (`uncertainty`), which computes the uncertainty of each "object" . For instance, you might want to create three components with some characteristics defining them and a fucntion describing each one's uncertainty:

In [1]:
# Component 1:
class component1():
    def __init__(self, name, param1, param2, comp1_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.uncertainty       = comp1_uncertainty # python module computing uncertainty of componentA. Defined by the user.

# Component 2:
class component2():
    def __init__(self, name, param1, param2, comp2_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.uncertainty       = comp2_uncertainty # python module computing uncertainty of componentB. Defined by the user.

# Component 3:
class component3():
    def __init__(self, name, param1, param2,param3 ,comp3_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.param3            = param3
        self.uncertainty       = comp3_uncertainty # python module computing uncertainty of componentC. Defined by the user.

Then, this "virtual objects", representing the actual lidar components, are encapsulated into another python class representing the actual lidar module. So, having the components we can create 2 different modules; `module1` and `module2`, each of them containing two and one of the components created above, respectively. They also contain a uncertainty function which computes uncertainty expansion among the lidar components we include in the module.


In [2]:
# Module 1:
class module1():
    def __init__(self,name,component1,component2,mod1_uncertainty):
        self.name             = name
        self.component1       = Component1 # here should have the name of the component instance
        self.component2       = Component2 # here should have the name of the component instance
        self.uncertainty      = mod1_uncertainty # python module computing uncertainty of module1. Defined by the user.

# Module 2:
class module2():
    def __init__(self,name,component1,mod2_uncertainty):
        self.name             = name
        self.component1       = Component3 # here should have the name of the component instance
        self.uncertainty      = mod2_uncertainty # python module computing uncertainty of module2. Defined by the user. GUM

Once the modules are built up, we can encapsulate them into the lidar class, representing the lidar device:

In [3]:
# Lidar object:
class lidar():
    def __init__(self, name,module1, module2, lidar_uncertainty):
        self.name        = name
        self.module1     = module1 # here should have the name of the module instance
        self.module2     = module2 # here should have the name of the module instance
        self.uncertainty = lidar_uncertainty # python module computing uncertainty of lidar device. Defined by the user. GUM

## Instantianting process (*Qlunc_Instantiate.py*):
We've created so far an empty structure of a lidar. Now we fill in the components, modules and the lidar itself by giving values to the variables and instantiating the corresponding classes.

All the parameters values are inputted through a yaml file (*Qlunc_inputs.yml*). We do this manually here to show the process:
### Components

In [4]:
Component1=component1(name              = 'Comp1',
                      param1            = 'parameter1_Comp1',
                      param2            = 'parameter2_Comp1',
                      comp1_uncertainty = 'unc_func_Comp1')

Component2=component2(name              = 'Comp2',
                      param1            = 'parameter1_Comp2',
                      param2            = 'parameter2_Comp2',
                      comp2_uncertainty = 'unc_func_Comp2')

Component3=component3(name              = 'Comp3',
                      param1            = 'parameter1_Comp3',
                      param2            = 'parameter2_Comp3',
                      param3            = 'parameter3_Comp3',
                      comp3_uncertainty = 'unc_func_Comp3')

In [5]:
print(Component1.name)
print(Component1.param1)
print(Component2.param2)
print(Component3.uncertainty)


Comp1
parameter1_Comp1
parameter2_Comp2
unc_func_Comp3


### Modules

In [6]:
Module1 = module1(name             = 'Mod1',
                  component1       = Component1,
                  component2       = Component2,
                  mod1_uncertainty ='unc_func_Mod1')
Module2 = module2(name             = 'Mod2',
                component1         = Component3,
                mod2_uncertainty   = 'unc_func_Mod2')

In [7]:
print(Module1.component1.param1)
print(Module2.component1.param3)

parameter1_Comp1
parameter3_Comp3


### Lidar

In [8]:
Lidar= lidar(name='Caixa1', module1=Module1,module2=Module2,lidar_uncertainty='Lidar_unc_func')

In [9]:
print(Lidar.uncertainty)

Lidar_unc_func


## How do we compute uncertainties
Import the functions calculating uncertainties:

In [10]:
import os
#os.chdir('../')
# importing  uncertainty functions
#import UQ_Functions.UQ_Photonics_Classes as uphc,UQ_Functions.UQ_Optics_Classes as uopc, UQ_Functions.UQ_Power_Classes as upwc,UQ_Functions.UQ_Lidar_Classes as ulc
#from Utils.Qlunc_ImportModules import *

In [11]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 0C54-9785

 Directory of C:\Users\fcosta\SWE_LOCAL\GIT_Qlunc\Tutorials

01.05.2021  22:10    <DIR>          .
01.05.2021  22:10    <DIR>          ..
01.05.2021  22:05    <DIR>          .ipynb_checkpoints
01.05.2021  22:06            15.590 Qlunc_Instantiate_Tutorial.py
30.03.2021  02:01             1.045 readme.md
01.05.2021  22:10            15.324 Tutorial0.ipynb
01.05.2021  15:04           210.033 Tutorial1.ipynb
01.05.2021  15:20            87.280 Tutorial2.ipynb
30.04.2021  13:39            10.273 WorkingExample_Tutorial.py
01.05.2021  15:02             4.941 yaml_inputs_file_1.yml
01.05.2021  20:49             4.950 yaml_inputs_file_2_1.yml
01.05.2021  14:47             4.953 yaml_inputs_file_2_2.yml
               9 File(s)        354.389 bytes
               3 Dir(s)  157.167.747.072 bytes free


In [12]:
# Obtain data from .yaml file:
os.chdir('../Utils')

In [13]:
from Qlunc_ImportModules import *
os.chdir('../')

In [ ]:
!dir

In [16]:
# Execute Qlunc_Classes.py (creating classes for lidar 'objects'):
exec(open('./Tutorials/Qlunc_Instantiate_Tutorial.py').read())

> <string>(29)<module>()



ipdb>  c


FileNotFoundError: [Errno 2] No such file or directory: './Tutorials/yaml_inputs_file1.yml'

In [ ]:
Qlunc_yaml_inputs['Main_directory']

### Photodetector
Different noise contributions are taken into account. Shot noise, thermal noise and dark current noise are computed. An option including TIA noise is also given.